In [1]:
!pip install -q gradio
!pip install -q torch
!pip install -q Pillow
!pip install -q diffusers
!pip install -q transformers
!pip install -q accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.1/305.1 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 6.6 MB/s e

In [2]:
import math
import random
import gradio as gr
import torch
from PIL import Image, ImageOps
from diffusers import StableDiffusionInstructPix2PixPipeline

model_id = "timbrooks/instruct-pix2pix"

def main():
    pipe = StableDiffusionInstructPix2PixPipeline.from_pretrained(model_id, torch_dtype=torch.float16, safety_checker=None).to("cuda")

    def generate(
        input_image: Image.Image,
        custom_Instructions: str,
        steps: int,
        randomize_seed: bool,
        seed: int,
        randomize_cfg: bool,
        text_cfg_scale: float,
        image_cfg_scale: float,
        artist_Gogh: bool,
        artist_Picasso: bool,
        artist_Warhol: bool,
        artist_Dali: bool,
        style_INDEX: int,
    ):
        if input_image is None:
            return [None, None, None, None]

        # Check if there's a custom instruction
        selected_Artists = [custom_Instructions] if custom_Instructions else []

        # Check what artists have been selected
        selected_Artists += [artist for artist, selected in zip(["Van Gogh", "Picasso", "Andy Warhol", "Salvador Dali"], [artist_Gogh, artist_Picasso, artist_Warhol, artist_Dali]) if selected]
        artist_names_String = " AND ".join(selected_Artists)

        # Extra Pix2Pix settings for randomization
        seed = random.randint(0, 100000) if randomize_seed else seed
        text_cfg_scale = round(random.uniform(6.0, 9.0), ndigits=2) if randomize_cfg else text_cfg_scale
        image_cfg_scale = round(random.uniform(1.2, 1.8), ndigits=2) if randomize_cfg else image_cfg_scale

        width, height = input_image.size
        factor = 512 / max(width, height)
        factor = math.ceil(min(width, height) * factor / 64) * 64 / min(width, height)
        width = int((width * factor) // 64) * 64
        height = int((height * factor) // 64) * 64
        input_image = ImageOps.fit(input_image, (width, height), method=Image.Resampling.LANCZOS)

        # Check what style has been selected
        style = "Vibrant painting" if style_INDEX == 0 else "Black and white drawing"

        # Create the final instruction
        instruction = f"{custom_Instructions}{' and ' if custom_Instructions else ''} make into a {style} in the style of {artist_names_String}{' and ' if artist_names_String else ''}"

        # Set guidance scale
        guidance_scale = 4.0
        image_guidance_scale = 1.0

        # Generate image
        generator = torch.manual_seed(seed)
        edited_image = pipe(
            instruction, image=input_image,
            guidance_scale=guidance_scale, image_guidance_scale=image_guidance_scale,
            num_inference_steps=steps, generator=generator,
        ).images[0]
        return [seed, text_cfg_scale, image_cfg_scale, edited_image]

    # Create title
    with gr.Blocks() as demo:
        gr.HTML("""<h1 style="font-weight: 900; margin-bottom: 7px;">
          MAIchelangelo 2.0
        </h1>
        <p>
        <br/>
        <a href="">
        <img style="margin-top: 0em; margin-bottom: 0em" src="" alt=""></a>
        <p/>
        <h2>Capture an image with your webcam and transform it into a painting or drawing in different art styles.</h2>""")

        # Webcam and output
        with gr.Row():
            input_image = gr.Image(label="Input Image", sources="webcam", type="pil", interactive=True)
            edited_image = gr.Image(label=f"Edited Image", type="pil", interactive=False)

        # Generate button, art style, and custom instructions
        with gr.Row():
            generate_button = gr.Button("Generate")
            style_Choice = gr.Radio(
                ["Painting", "Drawing"],
                value="Painting", # On by default
                type="index",
                label="Art Style",
                show_label=True,
                interactive=True,
            )
            custom_Instructions = gr.Textbox(lines=1, label="Custom Instructions (Write additional artist names or instructions)", interactive=True)

        # Artist selector
        gr.HTML('<div style="text-align: center; font-size: 20px;">Pick artist styles for your image. Multiple can be selected at the same time.</div>')
        with gr.Row():
            artist_Gogh = gr.Checkbox(label="Van Gogh", interactive=True, value=False)
            artist_Picasso = gr.Checkbox(label="Picasso", interactive=True, value=True) # On by default
            artist_Warhol = gr.Checkbox(label="Andy Warhol", interactive=True, value=False)
            artist_Dali = gr.Checkbox(label="Salvador Dali", interactive=True, value=False)

        # Extra Pix2Pix settings
        with gr.Row():
            steps = gr.Number(value=50, precision=0, label="Steps", interactive=True, visible=False)
            randomize_seed = gr.Radio(
                ["Fix Seed", "Randomize Seed"],
                value="Randomize Seed",
                type="index",
                show_label=False,
                interactive=True,
                visible=False,
            )
            seed = gr.Number(value=1371, precision=0, label="Seed", interactive=True, visible=False)
            randomize_cfg = gr.Radio(
                ["Fix CFG", "Randomize CFG"],
                value="Fix CFG",
                type="index",
                show_label=False,
                interactive=True,
                visible=False,
            )
            text_cfg_scale = gr.Number(value=16.5, label=f"Text CFG", interactive=True, visible=False)
            image_cfg_scale = gr.Number(value=1.5, label=f"Image CFG", interactive=True, visible=False)

        # Generate button
        generate_button.click(
            fn=generate,
            inputs=[
                input_image,
                custom_Instructions,
                steps,
                randomize_seed,
                seed,
                randomize_cfg,
                text_cfg_scale,
                image_cfg_scale,
                artist_Gogh,
                artist_Picasso,
                artist_Warhol,
                artist_Dali,
                style_Choice,
            ],
            outputs=[seed, text_cfg_scale, image_cfg_scale, edited_image],
        )

    demo.launch(share=True)

if __name__ == "__main__":
    main()

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

...

model.safetensors:   0%|          | 0.00/492M [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://82687ef7ab8d20c877.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
